In [ ]:
#pip install --ignore-installed shiny

: 

In [1]:
from shiny import App, render, ui
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

In [3]:
pip install rpy2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
Using cached tzlocal-5.2-py3-none-any.whl (17 kB)
  Created wheel for rpy2: filename=rpy2-3.5.17-cp312-cp312-macosx_11_0_arm64.whl size=252699 sha256=3411ac3f001051b30b2585806aec95b47e96407543b0549de6b4a0407ddff555
  Stored in directory: /Users/brianyandell/Library/Caches/pip/wheels/fb/3c/a8/5c2ab28ca010aa26f09174d6ac337141721ee764f7b17c0953
Successfully built rpy2
Note: you may need to restart the kernel to use updated packages.


In [4]:
from rpy2 import robjects

# `faithful$eruptions` from R
eruptions = robjects.r['faithful'][0]

#eruptions = np.random.normal(loc=3.5, scale=1.0, size=272)

In [5]:
app_ui = ui.page_fluid(
    ui.input_select(
        "n_breaks",
        "Number of bins in histogram (approximate):",
        choices=[10, 20, 35, 50],
        selected=20
    ),
    ui.input_checkbox(
        "individual_obs",
        "Show individual observations",
        value=False
    ),
    ui.input_checkbox(
        "density",
        "Show density estimate",
        value=False
    ),
    ui.output_plot("main_plot"),
    ui.output_ui("bw_adjust")
)

In [12]:
def server(input, output, session):
    @output
    @render.plot
    def main_plot():
        fig, ax = plt.subplots()
        n_breaks = int(input.n_breaks())
        hist_data = np.histogram(eruptions, bins=n_breaks, density=True)
        ax.bar(hist_data[1][:-1], hist_data[0], width=np.diff(hist_data[1]), edgecolor='black', align='edge')

        if input.individual_obs():
            ax.plot(eruptions, np.zeros_like(eruptions), 'r|', markersize=10)

        if input.density():
            bw_adjust = input.bw_adjust()
            kde = gaussian_kde(eruptions, bw_method=bw_adjust)
            x_grid = np.linspace(min(eruptions), max(eruptions), 1000)
            ax.plot(x_grid, kde(x_grid), color='blue')

        ax.set_xlabel("Duration (minutes)")
        ax.set_title("Geyser eruption duration")
        return fig

    @output
    @render.ui
    def bw_adjust():
        if input.density():
            return ui.input_slider(
                "bw_adjust",
                "Bandwidth adjustment:",
                min=0.2,
                max=2.0,
                value=1.0,
                step=0.2
            )
        return None


In [13]:
app = App(app_ui, server)

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
if __name__ == "__main__":
    app.run()

INFO:     Started server process [27260]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:61347 - "GET / HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 61349) - "WebSocket /websocket/" [accepted]
INFO:     connection open
INFO:     connection closed
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [27260]
